In [ ]:
!pip install --upgrade --force-reinstall pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 

In [1]:
# Uninstall existing problematic packages
!pip uninstall -y numpy pandas pmdarima

# Install versions compatible with Colab's environment
!pip install numpy==1.26.4  # Version compatible with TensorFlow, Numba, etc.
!pip install pandas==2.2.2  # Version required by Google Colab
!pip install pmdarima==2.0.4  # Install pmdarima after numpy/pandas

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 21.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf-cu12 25.10.0 requires pandas<2.4.0dev0,>=2.0, which is not installed.
dopamine-rl 4.1.2 requires pandas>=0.24.2, which is not installed.
yfinance 0.2.66 requires pandas>=1.3.0, which is not installed.
bokeh 3.7.3 requires pandas>=1.2, which is not installed.
prophet 1.1.7 requires pandas>=1.0.4, which is not installed.
dask-cuda 25.10.0 requires pandas>=1.3, which is not installed.
gradio 5.49.1 requires pandas<3.0,>=1.0, which is not in

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 79.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.5 MB/s eta 0:00:00


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import random
import warnings
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima
import ipywidgets as widgets
from IPython.display import display, clear_output

warnings.filterwarnings("ignore")

# --- Helper Functions (No changes needed here) ---
coeff_pool = [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9]

def generate_arima_sample_with_cumsum(p_range=(0, 2), d_range=(0, 2), q_range=(0, 2), n_sample=500):
    while True:
        p, d, q = random.randint(*p_range), random.randint(*d_range), random.randint(*q_range)
        ar = [random.choice(coeff_pool) for _ in range(p)]
        ma = [random.choice(coeff_pool) for _ in range(q)]
        ar_params, ma_params = np.r_[1, -np.array(ar)] if p > 0 else [1], np.r_[1, np.array(ma)] if q > 0 else [1]
        arma_process = ArmaProcess(ar_params, ma_params)
        if arma_process.isstationary and arma_process.isinvertible:
            y = arma_process.generate_sample(nsample=n_sample)
            for _ in range(d): y = np.cumsum(y)
            return y, (p, d, q)

def plot_all_diffs(y):
    fig, axes = plt.subplots(3, 3, figsize=(18, 10))
    fig.suptitle("Time Series and its Differences (Guess ARIMA p,d,q)", fontsize=16)
    diffs = [y, np.diff(y), np.diff(np.diff(y))]
    titles = ["Original Series", "1st Difference", "2nd Difference"]
    for i, series in enumerate(diffs):
        axes[i, 0].plot(series)
        axes[i, 0].set_title(titles[i])
        plot_acf(series, ax=axes[i, 1], lags=20, title="Autocorrelation")
        plot_pacf(series, ax=axes[i, 2], lags=20, title="Partial Autocorrelation")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# --- Game Setup ---
try:
    total_rounds = int(input("How many models do you want to play? (1-15): "))
    if not 1 <= total_rounds <= 15: total_rounds = 5
except ValueError: total_rounds = 5
# --- Game Setup ---
total_rounds = 1
# --- Create Interactive Widgets with Hints ---
p_input = widgets.IntText(value=0, description='p (from PACF):', style={'description_width': 'initial'})
d_input = widgets.IntText(value=0, description='d (Stationarity):', style={'description_width': 'initial'})
q_input = widgets.IntText(value=0, description='q (from ACF):', style={'description_width': 'initial'})
submit_button = widgets.Button(description="Submit Guess")
output_area = widgets.Output() # To display plots and messages

# Game state variables
game_state = {
    'score': 0,
    'current_round': 1,
    'total_rounds': total_rounds,
    'true_params': None,
    'y_data': None
}

def start_new_round():
    """Generates a new plot and prepares for the next guess."""
    if game_state['current_round'] > game_state['total_rounds']:
        return # Game is over

    with output_area:
        clear_output(wait=True)
        print(f"--- Round {game_state['current_round']} of {game_state['total_rounds']} ---")
        y, params = generate_arima_sample_with_cumsum()
        game_state['true_params'] = params
        game_state['y_data'] = y
        plot_all_diffs(y)

def on_submit_button_clicked(b):
    """Handles the logic when the user clicks the submit button."""
    if game_state['current_round'] > game_state['total_rounds']:
        with output_area:
            clear_output(wait=True)
            print("The game is already over!")
        return

    # Get guesses from widgets
    p_guess, d_guess, q_guess = p_input.value, d_input.value, q_input.value
    true_p, true_d, true_q = game_state['true_params']

    # Scoring
    round_score = (p_guess == true_p) + (d_guess == true_d) + (q_guess == true_q)
    game_state['score'] += round_score

    with output_area:
        # Show feedback
        print(f"\nYou guessed: ARIMA({p_guess}, {d_guess}, {q_guess})")
        print(f"Correct model: ARIMA({true_p}, {true_d}, {true_q})")
        print(f"You scored {round_score} point(s) this round. Total score: {game_state['score']}")

    # Advance the round
    game_state['current_round'] += 1

    if game_state['current_round'] > game_state['total_rounds']:
        # Game Over
        with output_area:
            max_score = game_state['total_rounds'] * 3
            accuracy = (game_state['score'] / max_score) * 100
            print("\n--- GAME OVER ---")
            print(f"Final Score: {game_state['score']}/{max_score}")
            print(f"Accuracy: {accuracy:.1f}%")
        submit_button.disabled = True # Disable button after game ends
    else:
        # Start the next round
        start_new_round()

# --- Link Button to Function and Start Game ---
submit_button.on_click(on_submit_button_clicked)
display(widgets.HBox([d_input, p_input, q_input, submit_button]), output_area)
start_new_round()

How many models do you want to play? (1-15): 1


Output()

In [ ]:
# קוד: הורדת נתוני TA35 מ-Yahoo, ציור, ACF/PACF, והתאמת ARIMA
# דרישות: yfinance, pandas, matplotlib, statsmodels

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# --- 1) פרמטרים
ticker = "GOOG"                     # טיקר בבורסת תל אביב (כפי שביקשת)
start_date = "2020-01-01"             # מ־2020 עד היום
end_date = pd.Timestamp.today().date()  # היום (דינמי)

# --- 2) הורדת נתונים
df = yf.download(ticker, start=start_date, end=end_date, progress=False)

# בדיקה קצרה שהעמודה קיימת
if "Adj Close" in df.columns:
    series = df["Adj Close"].dropna()
else:
    # אם אין Adj Close — ניפול ל־Close
    series = df["Close"].dropna()

series.index = pd.to_datetime(series.index)  # וידוא אינדקס תאריכים

# --- 3) ציור הסדרה (מחיר סגירה מתואם)
plt.figure(figsize=(12, 4))
plt.plot(series)
plt.title(f"{ticker} - Adjusted Close ({start_date} → {end_date})")
plt.xlabel("Date")
plt.ylabel("Price")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# --- 4) ACF & PACF לניתוח ראשוני (lags ניתן לשנות)
lags = 40
plt.figure(figsize=(12, 4))
plot_acf(series, lags=lags, alpha=0.05)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 4))
plot_pacf(series, lags=lags, method="ywm")  # או method="ld" בהתאם ל־statsmodels
plt.tight_layout()
plt.show()

# --- 5) התאמת מודל ARIMA
# בדוגמה המקורית נבחר MA(1) -> order=(0,0,1)
order = (4, 0, 1)   # שנה לפי צורך: (p,d,q)
model = ARIMA(series, order=order)

# fit המודל
fit = model.fit()

# תוצאיות עיקריות
print("\n--- ARIMA fit summary ---")
print(fit.summary())

# --- 6) חיזוי פשוט קדימה (לדוגמא 20 ימים)
steps_ahead = 20
forecast_res = fit.get_forecast(steps=steps_ahead)
forecast_mean = forecast_res.predicted_mean
conf_int = forecast_res.conf_int()

# ציור היסטוריה + חיזוי + אינטרוול ביטחון
plt.figure(figsize=(12, 4))
plt.plot(series, label="Historical")
# שרטוט התחזית על ציר תאריכים רציף אחרי היום האחרון
forecast_index = pd.date_range(start=series.index[-1] + pd.Timedelta(days=1),
                               periods=steps_ahead, freq="B")  # "B" = ימי עסקים
plt.plot(forecast_index, forecast_mean, linestyle="--", label=f"Forecast {steps_ahead} steps")
plt.fill_between(forecast_index,
                 conf_int.iloc[:, 0],
                 conf_int.iloc[:, 1],
                 alpha=0.2, label="95% CI")
plt.title(f"{ticker} - ARIMA{order} forecast ({steps_ahead} steps)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# קוד: הורדת נתוני TA35 מ-Yahoo, ציור, ACF/PACF, והתאמת ARIMA עם מניפה 10%-90%
# דרישות: yfinance, pandas, matplotlib, statsmodels

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# --- 1) פרמטרים
ticker = "TA35.TA"
start_date = "2020-01-01"
end_date = pd.Timestamp.today().date()

# --- 2) הורדת נתונים
df = yf.download(ticker, start=start_date, end=end_date, progress=False)

if "Adj Close" in df.columns:
    series = df["Adj Close"].dropna()
else:
    series = df["Close"].dropna()

series.index = pd.to_datetime(series.index)

# --- 3) ציור הסדרה
plt.figure(figsize=(12, 4))
plt.plot(series)
plt.title(f"{ticker} - Adjusted Close ({start_date} → {end_date})")
plt.xlabel("Date")
plt.ylabel("Price")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# --- 4) ACF & PACF
lags = 40
plt.figure(figsize=(12, 4))
plot_acf(series, lags=lags, alpha=0.05)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 4))
plot_pacf(series, lags=lags, method="ywm")
plt.tight_layout()
plt.show()

# --- 5) התאמת ARIMA
order = (4, 1, 1)
model = ARIMA(series, order=order)
fit = model.fit()

print("\n--- ARIMA fit summary ---")
print(fit.summary())

# --- 6) חיזוי קדימה
steps_ahead = 20
forecast_res = fit.get_forecast(steps=steps_ahead)

# חישוב מניפה עם רמות ביטחון שונות
percentiles = [0.1, 0.25, 0.75, 0.9]  # 10%, 25%, 75%, 90%
forecast_index = pd.date_range(start=series.index[-1] + pd.Timedelta(days=1),
                               periods=steps_ahead, freq="B")

forecast_mean = forecast_res.predicted_mean
conf_int_10 = forecast_res.conf_int(alpha=0.20)  # 10%-90%
conf_int_50 = forecast_res.conf_int(alpha=0.50)  # 25%-75%

# --- 7) ציור מניפה + היסטוריה
plt.figure(figsize=(12, 4))
plt.plot(series, label="Historical")
plt.plot(forecast_index, forecast_mean, linestyle="--", label="Forecast")

# מניפה 10%-90%
plt.fill_between(forecast_index,
                 conf_int_10.iloc[:, 0],
                 conf_int_10.iloc[:, 1],
                 color="blue", alpha=0.15, label="10%-90%")

# מניפה 25%-75%
plt.fill_between(forecast_index,
                 conf_int_50.iloc[:, 0],
                 conf_int_50.iloc[:, 1],
                 color="blue", alpha=0.25, label="25%-75%")

plt.title(f"{ticker} - ARIMA{order} forecast with fan chart")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.show()

# --- 8) ציור נפרד: רק תחזית + מניפה
plt.figure(figsize=(12, 4))
plt.plot(forecast_index, forecast_mean, linestyle="--", color="red", label="Forecast")

# מניפה 10%-90%
plt.fill_between(forecast_index,
                 conf_int_10.iloc[:, 0],
                 conf_int_10.iloc[:, 1],
                 color="blue", alpha=0.15, label="10%-90%")

# מניפה 25%-75%
plt.fill_between(forecast_index,
                 conf_int_50.iloc[:, 0],
                 conf_int_50.iloc[:, 1],
                 color="blue", alpha=0.25, label="25%-75%")

plt.title(f"{ticker} - Forecast only (ARIMA{order})")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

# === 1. הורדת נתונים מ-Yahoo Finance ===
ticker = "GOOG"
start_date = "2020-01-01"
end_date = "2025-09-04"

df = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)
df = df[['Close']]
df['Close'] = df['Close'].ffill().bfill()

# === 2. הכנת סדרת הנתונים ===
y = df['Close'].values
history = [float(x) for x in y]

order = (3, 0, 4)
steps_ahead = 30

predictions = []
lower_bounds = []
upper_bounds = []

# === 3. חיזוי רקורסיבי עם תחום ביטחון ===
for t in range(steps_ahead):
    model = ARIMA(history, order=order)
    fit = model.fit()

    forecast_result = fit.get_forecast(steps=1)
    yhat = float(forecast_result.predicted_mean[0])

    conf_int = forecast_result.conf_int(alpha=0.05)  # 95% CI
    lower = float(conf_int[0, 0])  # Access elements using NumPy indexing
    upper = float(conf_int[0, 1])  # Access elements using NumPy indexing

    predictions.append(yhat)
    lower_bounds.append(lower)
    upper_bounds.append(upper)
    history.append(yhat)

    print(t, yhat, lower, upper)

# === 4. יצירת תאריכים עתידיים ===
future_dates = pd.bdate_range(start=df.index[-1] + pd.Timedelta(days=1), periods=steps_ahead)

# === 5. גרף היסטוריה + תחזית + מניפה ===
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Close'], label="מחיר היסטורי")
plt.plot(future_dates, predictions, color="red", linestyle="--", label="תחזית קדימה")
plt.fill_between(future_dates, lower_bounds, upper_bounds, color='pink', alpha=0.3, label='95% Confidence Interval')
plt.title("תחזית ARIMA (רקורסיבית) למדד ת״א 35 עם מניפה")
plt.xlabel("תאריך")
plt.ylabel("מחיר סגירה")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(future_dates, predictions, color="red", linestyle="--", label="תחזית קדימה")
plt.fill_between(future_dates, lower_bounds, upper_bounds, color='pink', alpha=0.3, label='80% Confidence Interval (10%-90%)')
plt.title(f"{ticker} - 30 ימי תחזית קדימה עם מניפה 10%-90%")
plt.xlabel("תאריך")
plt.ylabel("מחיר סגירה")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

# === 1. הורדת נתונים מ-Yahoo Finance ===
ticker = "TA35.TA"
start_date = "2022-01-01"
end_date = "2025-09-04"

df = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)
df = df[['Close']]
df['Close'] = df['Close'].ffill().bfill()

# === 2. הכנת סדרת הנתונים ===
y = df['Close'].values
order = (3, 0, 4)   # אפשר לשנות לפי הצורך
steps_ahead = 30

# === 3. אמידה דו-שלבית (Powell → L-BFGS-B) ===
# שלב ראשון – Powell
model = ARIMA(y, order=order)
fit_css = model.fit(method_kwargs={"optimizer": "powell"})
print("שלב ראשון (Powell) - AIC:", fit_css.aic)

# שלב שני – L-BFGS-B עם הערכים מהשלב הראשון
fit_mle = model.fit(start_params=fit_css.params, method_kwargs={"optimizer": "lbfgs"})
print("שלב שני (L-BFGS-B) - AIC:", fit_mle.aic)

# === 4. חיזוי קדימה עם תחום ביטחון ===
# === 4. חיזוי קדימה עם תחום ביטחון ===
forecast_result = fit_mle.get_forecast(steps=steps_ahead)
predictions = forecast_result.predicted_mean
conf_int = forecast_result.conf_int(alpha=0.05)  # 95% CI, כאן מחזיר numpy.ndarray

# שימוש ב-Numpy אינדקסינג
lower_bounds = conf_int[:, 0]
upper_bounds = conf_int[:, 1]


# === 5. יצירת תאריכים עתידיים ===
future_dates = pd.bdate_range(start=df.index[-1] + pd.Timedelta(days=1), periods=steps_ahead)

# === 6. גרף היסטוריה + תחזית + מניפה ===
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Close'], label="מחיר היסטורי")
plt.plot(future_dates, predictions, color="red", linestyle="--", label="תחזית קדימה (CSS-MLE בסגנון Powell→L-BFGS-B)")
plt.fill_between(future_dates, lower_bounds, upper_bounds, color='pink', alpha=0.3, label='95% Confidence Interval')
plt.title("תחזית ARIMA (Powell → L-BFGS-B) למדד GOOG")
plt.xlabel("תאריך")
plt.ylabel("מחיר סגירה")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# === 4. חיזוי קדימה עם תחומי ביטחון שונים ===
forecast_result = fit_mle.get_forecast(steps=steps_ahead)
predictions = forecast_result.predicted_mean

# תחום ביטחון 90%
conf_int_90 = forecast_result.conf_int(alpha=0.10)  # 90% CI
lower_90 = conf_int_90[:, 0]
upper_90 = conf_int_90[:, 1]

# תחום ביטחון 80%
conf_int_80 = forecast_result.conf_int(alpha=0.20)  # 80% CI
lower_80 = conf_int_80[:, 0]
upper_80 = conf_int_80[:, 1]

# === 5. יצירת תאריכים עתידיים ===
future_dates = pd.bdate_range(start=df.index[-1] + pd.Timedelta(days=1), periods=steps_ahead)

# === 6. גרף תחזית בלבד עם שני רווחי סמך ===
plt.figure(figsize=(12, 6))
plt.plot(future_dates, predictions, color="black", linestyle="--", linewidth=2, label="תחזית קדימה")

# רווח סמך 90%
plt.fill_between(future_dates, lower_90, upper_90, color='red', alpha=0.4, label='90% Confidence Interval')

# רווח סמך 80%
plt.fill_between(future_dates, lower_80, upper_80, color='orange', alpha=0.6, label='80% Confidence Interval')

plt.title("תחזית ARIMA (Powell → L-BFGS-B) עם תחומי ביטחון 90% ו-80%")
plt.xlabel("תאריך")
plt.ylabel("מחיר סגירה")
plt.legend()
plt.grid(True)
plt


In [ ]:
print("\n--- ARIMA fit summary ---")
print(fit.summary())


In [ ]:
%pip install pmdarima --upgrade --no-deps